# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
import polars as pl
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [ ]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
    df['signal_prices'] = 0
    df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [7]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [9]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
    df['exit_prices'] = 0
    df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [10]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [11]:
@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio


In [12]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [13]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [14]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [16]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [17]:
# def generate_52w_data(data_path):
#     df = pd.read_csv(data_path)
#     df['datetime'] = pd.to_datetime(df['datetime'])
#     df = df.iloc[-(7 * 60 * 24 * 52):]
#     df.sort_values('datetime', ascending=True, inplace=True)
#     df.reset_index(inplace=True, drop=True)

#     return df

In [ ]:
def generate_52w_data(data_path):
    
    df = pl.read_csv(data_path)
    df = df.with_columns(pl.col('datetime').str.to_datetime())
    # df = df.slice(-(7 * 60 * 24 * 52))
    df = df.sort('datetime', descending=False)

    return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [18]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\datasets/\all_data_1min.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()

,datetime,6e_open,6e_high,6e_low,6e_close,6e_volume,aapl_open,aapl_high,aapl_low,aapl_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2023-11-01 18:56:00,1.05600,1.05635,1.05590,1.05590,607,172.77,172.97,172.765,172.95,...,104.914062,104.945312,104.914062,104.937500,3727,106.812500,106.859375,106.796875,106.843750,11709
1,2023-11-01 18:58:00,1.05625,1.05640,1.05615,1.05630,396,173.10,173.27,173.060,173.25,...,104.937500,104.945312,104.921875,104.937500,3415,106.843750,106.859375,106.828125,106.859375,8757
2,2023-11-01 18:59:00,1.05630,1.05650,1.05605,1.05615,1569,173.24,173.24,173.090,173.19,...,104.937500,104.960938,104.929688,104.960938,13873,106.859375,106.875000,106.828125,106.875000,33414
3,2023-11-01 19:02:00,1.05650,1.05670,1.05640,1.05660,326,173.59,173.75,173.550,173.74,...,104.976562,105.015625,104.968750,105.007812,5523,106.906250,106.937500,106.890625,106.921875,7998
4,2023-11-01 19:03:00,1.05655,1.05675,1.05645,1.05665,327,173.72,173.73,173.650,173.72,...,105.000000,105.023438,105.000000,105.000000,4571,106.921875,106.953125,106.906250,106.906250,9335


## Loading strategies generated

In [ ]:
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (2).csv")

try:
    df_str = pl.read_csv(strategy_file_path)
except:
    df_str = pl.read_csv(strategy_file_path, separator=';')

df_str = df_str.filter((pl.col('fitness') < 0) & (pl.col('fitness') <= -1.5))
df_str = df_str.unique()
df_str = df_str.sort('fitness', descending=False)
# df_str = df_str.to_pandas()
df_str

## Running the tests over the strategies

In [ ]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    buy_exit_txt = row.exit_buy
    sell_signal_txt = row.sell
    sell_exit_txt = row.exit_sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_buy_sell_pnl, get_drawdowns, get_pnl, get_lag
from fitness.performance.helper_func import trading_signals_buy, trading_signals_sell, change_exit
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, buy_exit_idxs = trading_signals_buy(buy_signal={buy_signal_txt}, exit_signal={buy_exit_txt})
ell_idxs, sell_exit_idxs = trading_signals_sell(sell_signal={sell_signal_txt}, exit_signal={sell_exit_txt})
buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs = change_exit(buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs
if len(buy_idxs) == 0 or len(sell_idxs) == 0 or len(buy_exit_idxs) == 0 or len(sell_exit_idxs) == 0:
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    # pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    buy_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_exit_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    sell_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_exit_idxs)]
    buy_arr = get_pnl(buy_exit_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    sell_arr = get_pnl(sell_exit_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    all_arr = merge_buy_sell_pnl(buy_idxs, sell_idxs, buy_arr, sell_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        # df = df_52w.iloc[idx:idx+bars_per_5week, :]
        # df.reset_index(drop=True, inplace=True)
        df = df_52w.slice(idx, idx+bars_per_5week-1).to_pandas()

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        price_data['day_of_week'] = (df['datetime'].dt.dayofweek + 1).values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        equity_curve_arr = exec_dict['equity_curve_arr']
        equity_curve_dict[strategy_idx].append(equity_curve_arr)

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=False)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

In [22]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [23]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",55.302829,29.945791,55.303960,0.0
1,strategy2,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",55.302829,29.945791,55.304874,0.0
2,strategy3,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",55.302829,29.945791,0.000000,0.0
3,strategy4,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",55.302829,29.945791,55.302829,0.0
4,strategy5,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",55.302829,29.945791,0.000000,0.0
...,...,...,...,...,...,...,...
12871,strategy12872,"np.cos(get_lag(price_data['aav_high'], lag=5)[...","np.cos(get_lag(price_data['inj_open'], lag=2)[...",NaN,NaN,NaN,100.0
12872,strategy12873,"np.sqrt(get_lag(price_data['tia_volume'], lag=...","get_lag(price_data['gc_volume'], lag=1)[MAX_LA...",NaN,NaN,NaN,100.0
12873,strategy12874,(np.cos(price_data['xrp_close'][MAX_LAG:]) / n...,np.sin(price_data['pltr_high'][MAX_LAG:]) < 0.5,NaN,NaN,NaN,100.0
12874,strategy12875,(np.cos(price_data['ng_close'][MAX_LAG:]) < 50...,"(np.log(get_lag(price_data['dog_volume'], lag=...",NaN,NaN,NaN,100.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [24]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",71.798231,71.795922,71.803534,0.0
712,strategy713,"(np.cos(get_lag(price_data['fet_high'], lag=5)...",(price_data['es_close'][MAX_LAG:] / get_lag(pr...,68.104225,89.011709,89.013306,0.0
1095,strategy1096,"np.sin(get_lag(price_data['tia_high'], lag=1)[...","((np.sin(get_lag(price_data['googl_volume'], l...",99.035495,77.551314,100.000000,0.0
1886,strategy1887,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",100.000000,90.772695,71.117752,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",100.000000,90.772695,71.117161,0.0
...,...,...,...,...,...,...,...
12680,strategy12681,"(np.sqrt(get_lag(price_data['es_volume'], lag=...","(get_lag(price_data['zf_volume'], lag=5)[MAX_L...",85.296757,84.053197,97.651343,0.0
12738,strategy12739,"np.sin(get_lag(price_data['zn_high'], lag=1)[M...","((get_lag(price_data['meta_open'], lag=2)[MAX_...",65.388887,60.593970,81.277506,0.0
12743,strategy12744,"np.sin(get_lag(price_data['pltr_high'], lag=1)...","((get_lag(price_data['sui_close'], lag=2)[MAX_...",82.090041,92.582592,80.508509,0.0
12765,strategy12766,"(np.sqrt(get_lag(price_data['tia_close'], lag=...","get_lag(price_data['tia_volume'], lag=5)[MAX_L...",66.216386,99.389553,60.389311,0.0


## Showing exit testing results

In [25]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",60.360453,60.432567,48.743071,0.0
1,strategy2,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",60.360453,60.432567,50.324204,0.0
2,strategy3,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",60.360453,60.432567,49.287161,0.0
3,strategy4,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",60.360453,60.432567,49.774959,0.0
4,strategy5,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",60.360453,60.432567,49.431330,0.0
...,...,...,...,...,...,...,...
12871,strategy12872,"np.cos(get_lag(price_data['aav_high'], lag=5)[...","np.cos(get_lag(price_data['inj_open'], lag=2)[...",NaN,NaN,NaN,100.0
12872,strategy12873,"np.sqrt(get_lag(price_data['tia_volume'], lag=...","get_lag(price_data['gc_volume'], lag=1)[MAX_LA...",NaN,NaN,NaN,100.0
12873,strategy12874,(np.cos(price_data['xrp_close'][MAX_LAG:]) / n...,np.sin(price_data['pltr_high'][MAX_LAG:]) < 0.5,NaN,NaN,NaN,100.0
12874,strategy12875,(np.cos(price_data['ng_close'][MAX_LAG:]) < 50...,"(np.log(get_lag(price_data['dog_volume'], lag=...",NaN,NaN,NaN,100.0


### Showing strategies that have passed entry testing

In [27]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",60.350224,60.432567,50.283306,0.0
712,strategy713,"(np.cos(get_lag(price_data['fet_high'], lag=5)...",(price_data['es_close'][MAX_LAG:] / get_lag(pr...,60.362498,60.438706,49.740212,0.0
1095,strategy1096,"np.sin(get_lag(price_data['tia_high'], lag=1)[...","((np.sin(get_lag(price_data['googl_volume'], l...",60.335904,60.418244,49.260540,0.0
1886,strategy1887,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,49.601113,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.203522,0.0
...,...,...,...,...,...,...,...
12680,strategy12681,"(np.sqrt(get_lag(price_data['es_volume'], lag=...","(get_lag(price_data['zf_volume'], lag=5)[MAX_L...",60.358407,60.506231,48.832928,0.0
12738,strategy12739,"np.sin(get_lag(price_data['zn_high'], lag=1)[M...","((get_lag(price_data['meta_open'], lag=2)[MAX_...",NaN,NaN,NaN,100.0
12743,strategy12744,"np.sin(get_lag(price_data['pltr_high'], lag=1)...","((get_lag(price_data['sui_close'], lag=2)[MAX_...",60.342041,60.360950,48.899480,0.0
12765,strategy12766,"(np.sqrt(get_lag(price_data['tia_close'], lag=...","get_lag(price_data['tia_volume'], lag=5)[MAX_L...",60.364544,60.432567,49.953972,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [28]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",60.350224,60.432567,50.283306,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.203522,0.0
1891,strategy1892,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.208657,0.0
1896,strategy1897,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.609532,0.0
1912,strategy1913,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.098184,0.0
1920,strategy1921,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.041932,0.0
1922,strategy1923,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.000000,0.0
1937,strategy1938,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.144210,0.0
1948,strategy1949,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.123750,0.0
1962,strategy1963,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",60.350224,60.432567,50.096142,0.0


## Showing core testing results

In [29]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",66.666667,0.0
1,strategy2,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",66.666667,0.0
2,strategy3,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",66.666667,0.0
3,strategy4,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",66.666667,0.0
4,strategy5,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",66.666667,0.0
...,...,...,...,...,...
12871,strategy12872,"np.cos(get_lag(price_data['aav_high'], lag=5)[...","np.cos(get_lag(price_data['inj_open'], lag=2)[...",NaN,100.0
12872,strategy12873,"np.sqrt(get_lag(price_data['tia_volume'], lag=...","get_lag(price_data['gc_volume'], lag=1)[MAX_LA...",NaN,100.0
12873,strategy12874,(np.cos(price_data['xrp_close'][MAX_LAG:]) / n...,np.sin(price_data['pltr_high'][MAX_LAG:]) < 0.5,NaN,100.0
12874,strategy12875,(np.cos(price_data['ng_close'][MAX_LAG:]) < 50...,"(np.log(get_lag(price_data['dog_volume'], lag=...",NaN,100.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [30]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",66.666667,0.0
712,strategy713,"(np.cos(get_lag(price_data['fet_high'], lag=5)...",(price_data['es_close'][MAX_LAG:] / get_lag(pr...,90.909091,0.0
1886,strategy1887,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1888,strategy1889,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
...,...,...,...,...,...
12550,strategy12551,"get_lag(price_data['nvda_volume'], lag=4)[MAX_...","(get_lag(price_data['inj_volume'], lag=3)[MAX_...",81.818182,0.0
12570,strategy12571,(price_data['es_volume'][MAX_LAG:] / 1.7045) >...,"(np.cos(get_lag(price_data['aav_open'], lag=5)...",60.000000,0.0
12599,strategy12600,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['aav_volume'], lag=3)[MAX_L...",77.777778,0.0
12633,strategy12634,"(get_lag(price_data['aav_high'], lag=3)[MAX_LA...","get_lag(price_data['btc_volume'], lag=3)[MAX_L...",85.714286,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [31]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",66.666667,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1891,strategy1892,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1896,strategy1897,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1912,strategy1913,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1920,strategy1921,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1922,strategy1923,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1937,strategy1938,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1948,strategy1949,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0
1962,strategy1963,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",66.666667,0.0


## Showing performance

In [32]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
0,strategy1,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",3.0,231368.41872,66.105262,2.087223,100.001245,2.0,110849.886201,660.72647,2.678809e+07,0.0
1,strategy2,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",3.0,231368.41872,66.105262,2.087223,100.001245,2.0,110849.886201,660.72647,2.678809e+07,0.0
2,strategy3,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",3.0,231368.41872,66.105262,2.087223,100.001245,2.0,110849.886201,660.72647,2.678809e+07,0.0
3,strategy4,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",3.0,231368.41872,66.105262,2.087223,100.001245,2.0,110849.886201,660.72647,2.678809e+07,0.0
4,strategy5,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",3.0,231368.41872,66.105262,2.087223,100.001245,2.0,110849.886201,660.72647,2.678809e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12871,strategy12872,"np.cos(get_lag(price_data['aav_high'], lag=5)[...","np.cos(get_lag(price_data['inj_open'], lag=2)[...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
12872,strategy12873,"np.sqrt(get_lag(price_data['tia_volume'], lag=...","get_lag(price_data['gc_volume'], lag=1)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
12873,strategy12874,(np.cos(price_data['xrp_close'][MAX_LAG:]) / n...,np.sin(price_data['pltr_high'][MAX_LAG:]) < 0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
12874,strategy12875,(np.cos(price_data['ng_close'][MAX_LAG:]) < 50...,"(np.log(get_lag(price_data['dog_volume'], lag=...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [33]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",3.0,200192.093405,57.197741,8.879748,100.006191,2.0,22544.793419,669.790212,5.448196e+06,0.0
712,strategy713,"(np.cos(get_lag(price_data['fet_high'], lag=5)...",(price_data['es_close'][MAX_LAG:] / get_lag(pr...,11.0,355538.430292,101.582409,19.416458,100.013577,10.0,12937.054226,552.531156,1.206845e+06,0.0
1886,strategy1887,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1888,strategy1889,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,strategy12075,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","(get_lag(price_data['msft_volume'], lag=5)[MAX...",11.0,211122.136795,60.320611,29902.980084,120.580808,6.0,5.491989,289.878670,4.653228e+02,0.0
12228,strategy12229,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","(get_lag(price_data['amzn_volume'], lag=1)[MAX...",11.0,162840.365381,46.525819,8061.642311,166.163103,7.0,4.916103,304.455362,9.658487e+02,0.0
12570,strategy12571,(price_data['es_volume'][MAX_LAG:] / 1.7045) >...,"(np.cos(get_lag(price_data['aav_open'], lag=5)...",5.0,75009.855475,21.431387,11621.000392,159.035243,1.0,3.281397,410.329005,9.359066e+02,0.0
12599,strategy12600,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['aav_volume'], lag=3)[MAX_L...",9.0,258792.964244,73.940847,85182.222652,139.650895,6.0,3.133479,248.551782,3.454137e+02,0.0


### Showing performance for strategies that has passed all tests and have positive ROI

In [34]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",3.0,200192.093405,57.197741,8.879748,100.006191,2.0,22544.793419,669.790212,5.448196e+06,0.0
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1891,strategy1892,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1896,strategy1897,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1912,strategy1913,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1920,strategy1921,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1922,strategy1923,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1937,strategy1938,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1948,strategy1949,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0
1962,strategy1963,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",3.0,315099.549803,90.028443,65.039461,100.021829,2.0,4844.744149,455.171589,1.170785e+06,0.0


In [1]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [2]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [3]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [38]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob
0,strategy1,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
1,strategy2,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
2,strategy3,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
3,strategy4,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
4,strategy5,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['zn_volume'], lag=3)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12871,strategy12872,"np.cos(get_lag(price_data['aav_high'], lag=5)[...","np.cos(get_lag(price_data['inj_open'], lag=2)[...",NaN,NaN,NaN,NaN,NaN,NaN
12872,strategy12873,"np.sqrt(get_lag(price_data['tia_volume'], lag=...","get_lag(price_data['gc_volume'], lag=1)[MAX_LA...",NaN,NaN,NaN,NaN,NaN,NaN
12873,strategy12874,(np.cos(price_data['xrp_close'][MAX_LAG:]) / n...,np.sin(price_data['pltr_high'][MAX_LAG:]) < 0.5,NaN,NaN,NaN,NaN,NaN,NaN
12874,strategy12875,(np.cos(price_data['ng_close'][MAX_LAG:]) < 50...,"(np.log(get_lag(price_data['dog_volume'], lag=...",NaN,NaN,NaN,NaN,NaN,NaN


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [39]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
712,strategy713,"(np.cos(get_lag(price_data['fet_high'], lag=5)...",(price_data['es_close'][MAX_LAG:] / get_lag(pr...,NaN,NaN,NaN,NaN,NaN,NaN
1886,strategy1887,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1888,strategy1889,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12074,strategy12075,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","(get_lag(price_data['msft_volume'], lag=5)[MAX...",NaN,NaN,NaN,NaN,NaN,NaN
12228,strategy12229,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","(get_lag(price_data['amzn_volume'], lag=1)[MAX...",NaN,NaN,NaN,NaN,NaN,NaN
12570,strategy12571,(price_data['es_volume'][MAX_LAG:] / 1.7045) >...,"(np.cos(get_lag(price_data['aav_open'], lag=5)...",NaN,NaN,NaN,NaN,NaN,NaN
12599,strategy12600,"np.cos(get_lag(price_data['fet_high'], lag=5)[...","get_lag(price_data['aav_volume'], lag=3)[MAX_L...",NaN,NaN,NaN,NaN,NaN,NaN


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [40]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob
176,strategy177,"np.sin(get_lag(price_data['amzn_high'], lag=5)...","get_lag(price_data['aapl_volume'], lag=3)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1887,strategy1888,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1891,strategy1892,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1896,strategy1897,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1912,strategy1913,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1920,strategy1921,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1922,strategy1923,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1937,strategy1938,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1948,strategy1949,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN
1962,strategy1963,"(np.sqrt(get_lag(price_data['lin_volume'], lag...","get_lag(price_data['amzn_volume'], lag=2)[MAX_...",NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_last.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_last.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_last.csv', index=False)
final_perf_df.to_csv('testing_results/perf_last.csv', index=False)
final_mc_df.to_csv('testing_results/mc_last.csv', index=False)